In [ ]:
!pip install -qU "langchain[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-xxxx"

## Runtime & State

### node-style

In [ ]:
#dataclass



In [ ]:
def log_before_model():

    return None

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import LLMToolEmulator

agent = create_agent(
    model="gpt-5-mini",
    tools=[],


)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "제 이름이 뭐죠?"}]},
    context=Context(user_name="Jay")
)

### wrap-style

In [ ]:
from langchain.messages import HumanMessage, SystemMessage

def inject_user_name(request, handler):


In [ ]:
from langchain.agents.middleware import after_model

@after_model
def log_after_model(state, runtime) :
    print(f"모델 응답 완료 후 : {state["messages"][-1].content}")
    return None

In [ ]:
agent = create_agent(
    model="gpt-5-mini",
    tools=[search_travel_destination_tool],
    middleware=[inject_user_name, log_after_model],
    context_schema=Context
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "제 이름이 뭐죠?"}]},
    context=Context(user_name="Jay Pak")
)

### 실습

## Custom middleware

### node-style

In [ ]:
from langchain.agents.middleware import after_model
from langchain.messages import AIMessage

@before_model()
def validate_input(state, runtime):

    return None

In [ ]:
agent = create_agent(
    model="gpt-5-mini",
    middleware=[validate_input],
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "오늘의 암구호는 삼각대-자동차 입니다."}]},
)

### wrap-style

In [ ]:
from langchain_openai import ChatOpenAI

@wrap_model_call
def dynamic_model_selector(request, handler):
    # 최근 사용자의 입력 메시지 추출
    last_msg = request.messages[-1].content if request.messages else ""
    msg_len = len(last_msg)

    # 길이에 따라 모델 선택

    print(f"🔁 메시지 길이: {msg_len}, 선택된 모델: {model_name}")

    # request.model을 새로운 모델로 교체


    # 수정된 요청으로 LLM 호출


In [ ]:
agent = create_agent(
    model="gpt-5-mini",
    tools=[],
    middleware=[dynamic_model_selector],
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "안녕하세요"}]},
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "안녕하세요. 저는 Jay입니다."}]},
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "안녕하세요. 저는 Jay입니다. 인공지능에 관해 궁금한게 있어요."}]},
)

## Guardrails

#### Before agent Guardrails

In [ ]:
# 차단 및 제재 키워드 정의
forbidden_topics = {
    "cheating": ["답지", "정답 알려줘", "숙제 대신", "써줘", "베끼기"], # 부정행위 관련
    "distraction": ["롤", "게임", "유튜브", "아이돌", "웹툰"],         # 학습 방해 요소
    "harmful": ["담배", "술", "폭력", "싸움", "바보"]                 # 유해 콘텐츠
}

In [ ]:
from langchain.agents.middleware import before_agent

@before_agent(can_jump_to=["end"])
def education_guardrail(state, runtime):
    """
    학생의 질문 의도를 파악하여 교육적이지 않거나 부정행위가 의심될 경우,
    LLM(AI)에게 질문을 넘기지 않고 교육적인 멘트로 즉시 교정합니다.
    """

    # 1. 메시지 유효성 검사
    if not state["messages"]:
        return None

    last_message = state["messages"][-1]
    if last_message.type != "human":
        return None

    user_text = last_message.content

    # 2. 카테고리별 검사 로직
    # 단순히 막는 것을 넘어, '왜' 안되는지 카테고리별로 다른 피드백 주기

    # Case A: 부정행위 방지 (Cheating Prevention)
    # AI가 숙제를 통째로 해주는 것을 방지
    for keyword in forbidden_topics["cheating"]:
        if keyword in user_text:
            return {
                "messages": [{
                    "role": "assistant",
                    "content": "🚫 스스로 고민해봐야 실력이 늘어요! 정답을 바로 알려드리는 대신, 힌트를 드릴까요? 어떤 부분이 가장 어려운지 말해주세요."
                }],
                "jump_to": "end"
            }

    # Case B: 학습 집중 유도 (Focus Management)
    # 공부 중에 게임이나 딴짓 이야기를 하면 다시 공부로 유도
    for keyword in forbidden_topics["distraction"]:
        if keyword in user_text:
            return {
                "messages": [{
                    "role": "assistant",
                    "content": "⏰ 지금은 공부에 집중할 시간이에요! 딴짓은 쉬는 시간에 하고, 지금 풀고 있는 문제에 집중해볼까요?"
                }],
                "jump_to": "end"
            }

    # Case C: 유해 콘텐츠 차단 (Safety)
    # 교육 서비스의 브랜드 안전성(Brand Safety)을 위한 기능
    for keyword in forbidden_topics["harmful"]:
        if keyword in user_text:
            return {
                "messages": [{
                    "role": "assistant",
                    "content": "⚠️ 부적절한 대화 주제입니다. 바르고 고운 말을 사용해주세요."
                }],
                "jump_to": "end"
            }

    # 3. 통과 (Pass)
    # 위 조건들에 걸리지 않으면 정상적으로 AI 튜터(LLM)가 답변 생성
    return None

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[],
    middleware=[education_guardrail],
)

In [ ]:
agent.invoke({
    "messages": [{"role": "user", "content": "피타고라스의 정리가 이해가 안 돼. 설명해줘."}]
})

In [ ]:
agent.invoke({
    "messages": [{"role": "user", "content": "나 독후감 쓰기 귀찮은데 숙제 대신 써줘."}]
})

In [ ]:
agent.invoke({
    "messages": [{"role": "user", "content": "아 공부하기 싫다. 롤 관련 유튜브 영상이나 찾아줘"}]
})

In [ ]:
agent.invoke({
    "messages": [{"role": "user", "content": "아 너 왜이렇게 바보같냐..."}]
})

#### After agent Guardrails

In [ ]:
from langchain.chat_models import init_chat_model

safety_model = init_chat_model("gpt-5-nano")

In [ ]:
from langchain.agents.middleware import after_agent

@after_agent
def answer_leakage_guardrail(state, runtime) :
    """
    AI가 답변을 생성한 '직후', 사용자에게 보여주기 전에 내용을 검사.
    만약 AI가 문제의 정답을 직접적으로 말해버렸다면, 이를 감지하고 수정.
    """

    # 1. 메시지 유효성 검사
    if not state["messages"]: return None
    last_message = state["messages"][-1]

    # 마지막 메시지가 AI의 답변이 아니면 검사할 필요 없음
    if not isinstance(last_message, AIMessage):
        return None

    # 2. 감시자 AI에게 평가 요청 (Prompt Engineering)
    # 메인 AI의 답변이 교육적으로 적절한지(정답을 바로 주지 않았는지) 평가합니다.
    auditor_prompt = f"""
    당신은 엄격한 교육 감독관입니다.
    다음 '튜터의 답변'을 확인하세요.
    답변이 학생을 지도하지 않고 문제의 정답이나 전체 풀이를 직접적으로 제공한다면 'LEAKED'라고 답하세요.
    답변이 적절한 힌트나 설명을 제공한다면 'SAFE'라고 답하세요.

    튜터의 답변: {last_message.content}
    """

    result = safety_model.invoke([{"role": "user", "content": auditor_prompt}])

    # 3. 결과에 따른 개입 (Intervention)
    if "LEAKED" in result.content:
        print(f"🚨 [가드레일 발동] 정답 유출 감지됨! 답변을 수정합니다.")
        last_message.content = "앗, 제가 정답을 바로 말할 뻔했네요! 😅 정답보다는 푸는 방법을 먼저 생각해볼까요? 이 문제의 핵심 개념은..."

    return None

In [ ]:
agent = create_agent(
    model="gpt-5",
    tools=[],
    middleware=[answer_leakage_guardrail],
)

In [ ]:
agent.invoke({
    "messages": [{"role": "user", "content": "직각 삼각형 두 직각변의 길이가 3과 4라면 빗변의 길이가 뭐야? 이 문제 너무 어려워. 그냥 정답 알려줘."}]
})

In [ ]:
from langchain.agents.middleware import after_agent


@after_agent
def answer_leakage_guardrail(state, runtime):
    """
    AI가 답변을 생성한 '직후', 사용자에게 보여주기 전에 내용을 검사.
    만약 AI가 문제의 정답을 직접적으로 말해버렸다면, 이를 감지하고 수정.
    """

    # 1. 메시지 유효성 검사
    if not state["messages"]: return None
    last_message = state["messages"][-1]

    # 마지막 메시지가 AI의 답변이 아니면 검사할 필요 없음
    if not isinstance(last_message, AIMessage):
        return None

    # 2. 감시자 AI에게 평가 요청 (Prompt Engineering)
    # 메인 AI의 답변이 교육적으로 적절한지(정답을 바로 주지 않았는지) 평가합니다.
    auditor_prompt = f"""
    당신은 엄격한 교육 감독관입니다.
    다음 '튜터의 답변'을 확인하세요.
    답변이 학생을 지도하지 않고 문제의 정답이나 전체 풀이를 직접적으로 제공한다면 'LEAKED'라고 답하세요.
    답변이 적절한 힌트나 설명을 제공한다면 'SAFE'라고 답하세요.

    튜터의 답변: {last_message.content}
    """

    result = safety_model.invoke([{"role": "user", "content": auditor_prompt}])

    # 3단계: 교정 (Correction / Regeneration)
    if "LEAKED" in result.content:

        # 원래 사용자의 질문을 가져오기 (문맥 파악용) -> state["messages"][-2]가 보통 사용자 질문
        original_question = state["messages"][-2].content if len(state["messages"]) >= 2 else "사용자 질문 알 수 없음"

        # 교정 모델에게 "정답을 빼고 힌트로 바꿔라"고 지시
        correction_prompt = f"""
        당신은 친절한 AI 튜터입니다.

        절대 정답을 직접 말하지 말고, 학생이 스스로 생각할 수 있도록 유도하는 질문이나 핵심 개념(힌트)만 설명하세요.
        말투는 친절하게 해주세요.

        사용자 질문: {original_question}
        """

        # LLM을 다시 호출하여 새로운 답변 생성 (비용은 1회 더 발생하지만 품질 확보)
        corrected_response = safety_model.invoke([
            SystemMessage(content="당신은 소크라테스식 교육법을 사용하는 튜터입니다."),
            HumanMessage(content=correction_prompt)
        ])

        # 원래의 유출된 답변을 교정된 답변으로 덮어쓰기
        last_message.content = corrected_response.content

    return None


In [ ]:
agent = create_agent(
    model="gpt-5",
    tools=[],
    middleware=[answer_leakage_guardrail],
)

In [ ]:
agent.invoke({
    "messages": [{"role": "user", "content": "직각 삼각형 두 직각변의 길이가 3과 4라면 빗변의 길이가 뭐야? 이 문제 너무 어려워. 그냥 정답 알려줘."}]
})

#### Combine multiple Guardrails

In [ ]:
import re

@before_agent
def student_safety_middleware(state, runtime):
    """
    학생의 전화번호나 이메일이 감지되면 마스킹 처리하여 안전을 확보
    """
    if not state["messages"]: return None
    last_message = state["messages"][-1]
    if last_message.type != "human": return None

    content = last_message.content
    original_content = content # 로깅용

    # 전화번호 패턴 (010-XXXX-XXXX 또는 010XXXXXXXX 등)
    phone_pattern = r'01[016789]-?[0-9]{3,4}-?[0-9]{4}'
    # 이메일 패턴
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

    is_redacted = False

    if re.search(phone_pattern, content):
        content = re.sub(phone_pattern, '<PHONE_REDACTED>', content)
        is_redacted = True

    if re.search(email_pattern, content):
        content = re.sub(email_pattern, '<EMAIL_REDACTED>', content)
        is_redacted = True

    if is_redacted:
        print(f"🔒 [학생 보호] 개인정보가 감지되어 마스킹 처리했습니다.\n원본: {original_content}\n수정: {content}")
        # 내용을 수정하여 LLM에게 전달 (사용자에게 알릴 필요 없이 조용히 처리하거나, 시스템 메시지 추가 가능)
        last_message.content = content

    return None

In [ ]:
ESCALATION_KEYWORDS = ["왕따", "괴롭힘", "우울해", "학교 폭력", "상담 선생님", "사람 불러줘"]

@before_agent(can_jump_to=["end"])
def counseling_escalation_middleware(state, runtime) :
    """
    [Layer 3] 심리적 위기 상황이나 상담 요청이 감지되면 AI 답변을 멈추고 인간 상담사에게 알림을 보냅니다.
    """
    if not state["messages"]: return None
    last_message = state["messages"][-1]

    # 민감한 키워드가 포함되어 있는지 확인
    for keyword in ESCALATION_KEYWORDS:
        if keyword in last_message.content:
            print(f"✋ [상담 이관] 심각한 고민/요청 감지: {keyword}")

            # 여기서 실제로는 상담 교사에게 알림(Slack, Email 등)을 보내는 로직이 들어감
            # send_alert_to_teacher(last_message.content)

            return {
                "messages": [{
                    "role": "assistant",
                    "content": "학생, 많이 힘들었겠구나. 이 문제는 내가 답변하기보다는 전문 상담 선생님이 직접 듣고 도와주시는 게 좋을 것 같아. \n\n지금 바로 상담 선생님께 연결해 드렸으니 잠시만 기다려 줄래? 🍀 (상담실 연결 중...)"
                }],
                "jump_to": "end" # AI 답변 생성 중단
            }
    return None


In [ ]:
# 4중 방어막이 적용된 에이전트
agent = create_agent(
    model="gpt-5-nano",
    tools=[],
    middleware=[
        education_guardrail,             # Layer 1: 입력 필터 (규칙 - 딴짓/부정행위)
        student_safety_middleware,       # Layer 2: 개인정보 보호 (전화번호 마스킹)
        counseling_escalation_middleware,# Layer 3: 상담 이관 (휴먼 에스컬레이션)
        answer_leakage_guardrail         # Layer 4: 출력 필터 (모델 기반 교정)
    ],
)

In [ ]:
agent.invoke({
    "messages": [{"role": "user", "content": "저 수학 과외 구하고 싶어요. 제 번호 010-1234-5678로 연락 주세요."}]
})

In [ ]:
agent.invoke({
    "messages": [{"role": "user", "content": "나 요즘 학교에서 왕따 당하는 것 같아서 너무 우울해."}]
})

## Long-term Memory

### 장기 메모리 생성

In [ ]:
# store 생성

In [ ]:
user_id = "user_001"
application_context = "personal_assistant"

# namespace 정의

In [ ]:
store.put(
    {
        "facts": [
            "사용자는 커피보다 차를 선호함",
            "사용자는 매일 아침 6시에 일어남",
        ],
        "language": "Korean",
    },
)

In [ ]:
item = store.get(namespace, "memory_001")
print("저장된 메모리:", item.value)

In [ ]:
items = store.search(namespace)

In [ ]:
items

In [ ]:
store.put(
    namespace,
    "memory_002",
    {
        "facts": [
            "사용자는 그림 회화 작품을 좋아함",
            "빈센트 반 고흐의 작품을 특히 좋아함",
        ]
    },
)

In [ ]:
item = store.get(namespace, "memory_002")
print("저장된 메모리:", item.value)

In [ ]:
items = store.search(namespace)

In [ ]:
items

In [ ]:
from dataclasses import dataclass

# 실행 컨텍스트 정의 (누가 실행하는지 식별)
@dataclass
class Context:
    user_id: str
    app_name: str

In [ ]:
from langchain.agents.middleware import wrap_model_call
from langchain.messages import HumanMessage, SystemMessage

@wrap_model_call
def inject_memory(request, handler):
    current_user = request.runtime.context.user_id
    current_app = request.runtime.context.app_name
    memories = request.runtime.store.search((current_user, current_app))

    memory_content = "기록된 정보 없음"
    if memories:
        # 검색된 메모리들을 텍스트 변환
        extracted_facts = []
        for item in memories:
            if "facts" in item.value:
                extracted_facts.extend(item.value["facts"])
        memory_content = "\n- ".join(extracted_facts)

    system_message=f"사용자 관련 장기 메모리 : {memory_content}"
    request = request.override(system_prompt=system_message)
    return handler(request)

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    store=store,          # store 연결
    context_schema=Context,
    middleware=[inject_memory]  # 미들웨어 장착
)

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "나에 대해 알고있는 정보 알려줘"}]},
    context=Context(user_id="user_001", app_name="personal_assistant")
)

In [ ]:
response

### Tool 기반 장기 메모리 관리

In [ ]:
from dataclasses import dataclass

# 실행 컨텍스트 정의 (누가 실행하는지 식별)
@dataclass
class Context:
    user_id: str
    app_name: str

# Store 초기화
store = InMemoryStore()

In [ ]:
from typing import TypedDict

# LLM이 추출해야 할 정보의 구조를 정의
class UserInfo(TypedDict):


In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain.tools import tool, ToolRuntime

# Tool 정의: 사용자의 정보를 조회
@tool
def get_user_info(runtime) -> str:
    """
    현재 사용자의 정보 조회 (시스템 내부용 도구)
    """
    user_id = runtime.context.user_id
    app = runtime.context.app_name

    # 해당 네임스페이스의 모든 메모리 검색
    memories = runtime.store.search((user_id, app))

    if not memories:
        return "기록된 정보 없음"

    results = []
    for item in memories:
        # 저장된 데이터 구조(UserInfo)에 맞춰 필드 확인
        data = item.value
        # 저장할 때 사용한 Key들을 확인해서 문자열로 변환
        if "personal_information" in data:
            results.append(f"- 개인정보: {data['personal_information']}")
        if "preference" in data:
            results.append(f"- 선호도: {data['preference']}")

    return "\n".join(results) if results else "데이터 형식 불일치로 읽을 수 없음"

In [ ]:
import uuid

@tool
def save_user_info(user_info, runtime):
    """
    사용자의 정보를 저장하거나 업데이트
    """
    # 1. 실행 컨텍스트에서 user_id 가져오기
    user_id = runtime.context.user_id
    app = runtime.context.app_name
    store = runtime.store

    # 2. Store에 데이터 저장 (put(namespace, key, value))
    memory_key = str(uuid.uuid4())
    store.put((user_id, app),memory_key, user_info)

    return f"정보가 안전하게 저장되었습니다. (ID: {memory_key})"

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[get_user_info, save_user_info],
    store=store, # 에이전트에 store 연결
    context_schema=Context
)

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "내 이름은 이제 'Alice'야. 커피보단 차를 좋아해"}]},
    context=Context(user_id="user_001", app_name="personal_assistant")
)

In [ ]:
response

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "나에 대해 아는 정보 말해줘"}]},
    context=Context(user_id="user_001", app_name="personal_assistant")
)

In [ ]:
response